# 기간별 여러 채널 유튜브 정보 검색 프로그램

## 프로그램 설명
YouTube Data API를 사용하여 여러 채널의 정보를 기간별로 검색하고 분석하는 프로그램입니다.

## 주요 기능
- YouTube Data API를 통한 채널 정보 수집
- 여러 채널 동시 검색
- 기간별 데이터 분석
- 동영상 정보 수집 (제목, 조회수, 업로드일 등)
- 동영상 길이 포맷팅

## 사용 방법
1. YouTube Data API 키 설정
2. 프로그램 실행
3. 검색할 채널 ID 입력
4. 기간 설정
5. 자동으로 채널 및 동영상 정보 수집

## 필요 라이브러리
- requests: API 요청
- pandas: 데이터 처리
- isodate: 날짜/시간 처리

## 주의사항
- YouTube Data API 키가 필요합니다
- API 사용량 제한이 있을 수 있습니다 (일일 할당량 확인 필요)
- API 키 유효성 검사 기능 포함

In [1]:
# 기간별 여러 채널 유튜브 정보 검색

import requests
import pandas as pd
from datetime import datetime
import isodate
import os  # os 모듈을 불러옵니다

def format_duration(duration):
    parsed_duration = isodate.parse_duration(duration)
    hours, remainder = divmod(parsed_duration.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

def check_api_key(api_key):
    test_url = f"https://www.googleapis.com/youtube/v3/channels?key={api_key}&part=id&id=UCBR8-60-B28hp2BmDPdntcQ"  # Google Developers channel for testing
    try:
        response = requests.get(test_url)
        response.raise_for_status()
        print("API 키가 유효합니다.")
    except requests.exceptions.HTTPError as e:
        if response.status_code == 403:
            print("API 키의 요청 한도 초과 또는 접근 권한 오류입니다.")
        else:
            print(f"API 키가 유효하지 않습니다. 오류: {e}")
    except requests.exceptions.RequestException as e:
        print(f"API 키 검증 실패: {e}")

def fetch_youtube_data(api_key, channel_ids):
    print("데이터 수집 중...")
    start_date = "2024-04-01T00:00:00Z"
    end_date = datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ')
    search_url_template = (
        'https://www.googleapis.com/youtube/v3/search?key={}&part=snippet&maxResults=50'
        '&type=video&order=date&publishedAfter={}&publishedBefore={}&channelId={}'
    )
    video_details = []
    video_index = 1  # 비디오 인덱스 초기화

    for channel_id in channel_ids:
        print(f"채널 {channel_id}에 대한 비디오 데이터 수집 시작...")
        search_url = search_url_template.format(api_key, start_date, end_date, channel_id)
        next_page_token = None

        while True:
            if next_page_token:
                search_url += f'&pageToken={next_page_token}'

            try:
                search_response = requests.get(search_url)
                search_response.raise_for_status()
            except requests.exceptions.RequestException as e:
                print(f"검색 결과 가져오기 실패: {e}")
                break

            search_results = search_response.json()
            next_page_token = search_results.get('nextPageToken')
            
            for item in search_results.get('items', []):
                video_id = item['id']['videoId']
                video_url = f"https://www.youtube.com/watch?v={video_id}"
                video_details_url = f'https://www.googleapis.com/youtube/v3/videos?id={video_id}&key={api_key}&part=contentDetails,statistics,snippet'
                try:
                    video_details_response = requests.get(video_details_url)
                    video_details_response.raise_for_status()
                except requests.exceptions.RequestException as e:
                    print(f"비디오 상세 정보 가져오기 실패: {e}")
                    continue

                video_info = video_details_response.json()
                if 'items' in video_info and video_info['items']:
                    video_data = video_info['items'][0]
                    print(f"비디오 '{video_data['snippet']['title']}' 정보 수집 중...")
                    video_details.append({
                        'Index': video_index,
                        'Title': video_data['snippet']['title'],
                        'Channel ID': channel_id,
                        'Channel Name': video_data['snippet']['channelTitle'],
                        'Duration': format_duration(video_data['contentDetails']['duration']),
                        'Views': int(video_data['statistics'].get('viewCount', 0)),
                        'Comments': int(video_data['statistics'].get('commentCount', 0)),
                        'Video URL': video_url,
                        'Thumbnail URL': video_data['snippet']['thumbnails']['high']['url'],
                        'Tags': ', '.join(video_data['snippet'].get('tags', [])),
                        'Published Date': video_data['snippet']['publishedAt'][:10]  # YYYY-MM-DD 형식으로 저장
                    })
                    video_index += 1  # 인덱스 증가

            if not next_page_token:
                print(f"채널 {channel_id}에 대한 데이터 수집 완료.")
                break

    return video_details

# 환경변수에서 API 키 가져오기
api_key = os.getenv('YOUTUBE_API_KEY')
if not api_key:
    raise ValueError("환경 변수에서 API 키를 찾을 수 없습니다.")

check_api_key(api_key)  # API 키 유효성 및 요청 한도 체크
channel_ids_input = input("채널 ID를 입력하세요 (쉼표로 구분): ")
channel_ids = [id.strip() for id in channel_ids_input.split(',')]
file_name = input("엑셀 파일 이름을 입력하세요: ")

# 데이터 추출
videos = fetch_youtube_data(api_key, channel_ids)

# 엑셀 파일로 저장
if videos:
    try:
        print("엑셀 파일로 저장 중...")
        df = pd.DataFrame(videos)
        today_date = datetime.now().strftime("%Y-%m-%d")
        file_path = f"{file_name}_{today_date}.xlsx"
        df.to_excel(file_path, index=False)
        print(f"데이터가 {file_path}에 저장되었습니다.")
    except Exception as e:
        print(f"엑셀 파일 저장 실패: {e}")
else:
    print("저장할 데이터가 없습니다.")



API 키가 유효합니다.
채널 ID를 입력하세요 (쉼표로 구분): UCZ6UHYBQFBe14WUgxlgmYfg
엑셀 파일 이름을 입력하세요: 오빠두최신
데이터 수집 중...
채널 UCZ6UHYBQFBe14WUgxlgmYfg에 대한 비디오 데이터 수집 시작...
비디오 '워드 고수만 아는, Tab 키의 숨겨진 기능⚡#shorts #직장인꿀팁' 정보 수집 중...
비디오 '초간단 엑셀 '와플 차트', 1분 안에 만드는 법⚡️#shorts #직장인엑셀' 정보 수집 중...
비디오 '엑셀을 이용한 마케팅 분석의 모든 것! | 고객 리뷰 & 구매 연관성 분석 【기프티쇼 마케터 초청】' 정보 수집 중...
비디오 '엑셀 머리글/요약행 고정하는 쉬운 방법⚡#직장인꿀팁 #shorts' 정보 수집 중...
비디오 '엑셀 줄바꿈 나누기, 단축키로 1초 안에 해결하는 법⚡️#shorts' 정보 수집 중...
비디오 '[원데이 챌린지] 엑셀 파워쿼리, 초보자 맞춤 3시간 총정리 특강 | 이 영상으로 마스터하세요!' 정보 수집 중...
비디오 '엑셀 자동 입력 보고서, 1분 안에 만드는 방법⚡#shorts' 정보 수집 중...
비디오 '엑셀 → QR 코드 만들기, 모든 버전 가능! 100% 무료 해결법⚡#shorts #엑셀' 정보 수집 중...
비디오 '2024년 필수! 엑셀 핵심 기능 30개 총정리 | 초보자를 위한 실전 스킬 모음zip' 정보 수집 중...
비디오 '엑셀 사용자라면 반드시 알아야 할 필수 설정 3가지 #shorts #엑셀' 정보 수집 중...
비디오 '엑셀 셀 병합 문제, 고민 끝! | 직장인을 위한 초간단 완벽 해결법 (클릭 10번이면 됩니다!)' 정보 수집 중...
비디오 '엑셀 반복 작업을 200% 줄여주는 필수 단축키 | 'F4' 이렇게 사용해보세요! #shorts' 정보 수집 중...
비디오 '엑셀과 코파일럿으로 바탕화면을 완벽하게 정리하는 법 | 폴더 구성까지, 3분이면 됩니다!' 정보 수집 중...
비디오 '엑셀 두 범위 비교, 단축키 1초 해결법⚡️